In [3]:
# log regression
# feature: start, end, day
# result: duration

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("../data/yellow_tripdata_2020-04.csv", nrows=6000000)

train.info()
train['tpep_pickup_datetime'] = pd.to_datetime(train['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
train['tpep_dropoff_datetime'] = pd.to_datetime(train['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
train['trip_duration'] = (train['tpep_dropoff_datetime'] - train['tpep_pickup_datetime']).dt.seconds
train['PULocationID'].fillna(-1, inplace=True)
train['DOLocationID'].fillna(-1, inplace=True)



/var/folders/wg/62v23f191zsbtpgk23vhgv2h0000gn/T/ipykernel_59455/1274987493.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/yellow_tripdata_2020-04.csv", nrows=6000000)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238073 entries, 0 to 238072
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               238073 non-null  int64  
 1   tpep_pickup_datetime   238073 non-null  object 
 2   tpep_dropoff_datetime  238073 non-null  object 
 3   passenger_count        218493 non-null  float64
 4   trip_distance          238073 non-null  float64
 5   RatecodeID             218493 non-null  float64
 6   store_and_fwd_flag     218493 non-null  object 
 7   PULocationID           238073 non-null  int64  
 8   DOLocationID           238073 non-null  int64  
 9   payment_type           238073 non-null  int64  
 10  fare_amount            238073 non-null  float64
 11  extra                  238073 non-null  float64
 12  mta_tax                238073 non-null  float64
 13  tip_amount             238073 non-null  float64
 14  tolls_amount           238073 non-nu

In [11]:

#X_train = train[['PULocationID', 'DOLocationID']]
#y_train = train[['trip_duration']]
#model_y = train[['total_amount']]
#X_test = df_05[['PULocationID', 'DOLocationID']]
#y_test = df_05[['trip_duration']]

from sklearn.model_selection import train_test_split

X=train.drop(["fare_amount"],axis=1)
y=train["fare_amount"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=123)#test_size is the proportion of data that is to be kept aside for validation


In [12]:
X_train = X_train[['PULocationID','DOLocationID']]
X_test = X_test[['PULocationID','DOLocationID']]


In [13]:
from sklearn.metrics import mean_squared_error

avg_fare=round(np.mean(y_train),2) #11.31
baseline_pred=np.repeat(avg_fare,y_test.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :",baseline_rmse)

Basline RMSE of Validation data : 11.443153287956056


In [14]:
# linear regression
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
lm_train_rmse=np.sqrt(mean_squared_error(lm.predict(X_train), y_train))
lm_variance=abs(lm_train_rmse - lm_rmse)
print("Test RMSE for Linear Regression is ",lm_rmse)
print("Train RMSE for Linear Regression is ",lm_train_rmse)
print("Variance for Linear Regression is ",lm_variance)

Test RMSE for Linear Regression is  11.398179107965786
Train RMSE for Linear Regression is  11.785619201795623
Variance for Linear Regression is  0.38744009382983613


In [15]:
# random forest
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state = 883,n_jobs=-1)
rf.fit(X_train,y_train)
rf_pred= rf.predict(X_test)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ",rf_rmse)

RMSE for Random Forest is  6.955823172265283


In [16]:
# LightGBM
import lightgbm as lgb

train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':31, 'num_trees':5000,'objective':'regression'}
param['metric'] = 'l2_root'
num_round=5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=10,verbose_eval=20, early_stopping_rounds=20,stratified=False)
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM is ",lgb_rmse)

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:573: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009784 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 2
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002367 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 2
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 2
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, th

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Start training from score 11.660562
[LightGBM] [Info] Start training from score 11.653872
[LightGBM] [Info] Start training from score 11.664505
[LightGBM] [Info] Start training from score 11.674088
[LightGBM] [Info] Start training from score 11.663011
[LightGBM] [Info] Start training from score 11.663297
[LightGBM] [Info] Start training from score 11.666338
[LightGBM] [Info] Start training from score 11.654494
[LightGBM] [Info] Start training from score 11.671579
[20]	cv_agg's rmse: 9.96166 + 0.714707
[40]	cv_agg's rmse: 9.58573 + 0.722929
[60]	cv_agg's rmse: 9.37474 + 0.738957
[80]	cv_agg's rmse: 9.22168 + 0.750375
[100]	cv_agg's rmse: 9.10347 + 0.756657
[120]	cv_agg's rmse: 9.01345 + 0.764549
[140]	cv_agg's rmse: 8.93757 + 0.767932
[160]	cv_agg's rmse: 8.86325 + 0.770781
[180]	cv_agg's rmse: 8.80171 + 0.77193
[200]	cv_agg's rmse: 8.74453 + 0.76814
[220]	cv_agg's rmse: 8.69523 + 0.77063
[240]	cv_agg's rmse: 8.65296 + 0.772102
[260]	cv_agg's rmse: 8.61335 + 0.776143
[

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 178554, number of used features: 2
[LightGBM] [Info] Start training from score 11.660754
RMSE for Light GBM is  7.170031094221064
